# [Students] Shopee Code League - Order Brushing.ipynb

In [205]:
import pandas as pd
from pandas.tseries.offsets import DateOffset
import csv
from tqdm import tqdm

In [2]:
df_file = pd.read_csv("order_brush_order.csv")
df_file['event_time'] = pd.to_datetime(df_file['event_time'])
df_file=df_file.sort_values(by=['shopid','event_time'])
df_file.head(10)

,orderid,shopid,userid,event_time
59,31086409141107,10009,196962305,2019-12-27 03:06:50
76187,31144571933461,10051,2854032,2019-12-27 19:16:11
9055,31254979546679,10051,48600461,2019-12-29 01:56:19
188513,31195675919209,10061,168750452,2019-12-28 09:27:55
174620,31205132327893,10061,194819216,2019-12-28 12:05:32
200439,31412115824794,10061,130633421,2019-12-30 21:35:15
145426,31431527100615,10061,62464559,2019-12-31 02:58:48
167859,31075686185309,10084,4401933,2019-12-27 00:08:06
178051,31077155357404,10084,13837190,2019-12-27 00:32:36
91837,31079024994425,10084,39828049,2019-12-27 01:03:44


In [3]:
# Berfungsi untuk mendapatkan daftar shopid
list_shopid = df_file['shopid'].unique()

In [4]:
def get_df_by_shopid(key_shopid, df):
    """Memperoleh dataframe berdasarkan shopid

    Parameters 
    ----------
    key_shopid : str
    df : DataFrame

    Returns
    -------
    DataFrame
        dataframe sesuai dengan shopid
    """
    df_shopid = df[df['shopid']==key_shopid]
    return df_shopid

get_df_by_shopid(list_shopid[25], df_file).head()

,orderid,shopid,userid,event_time
166922,31076890562912,10311,5314183,2019-12-27 00:28:11
97644,31078750586751,10311,48188032,2019-12-27 00:59:10
167900,31106297734012,10311,86593046,2019-12-27 08:38:17
121781,31113446916111,10311,1223238,2019-12-27 10:37:26
51555,31114127034746,10311,15372944,2019-12-27 10:48:48


In [5]:
def get_df_in_a_hour_from_series(s_index, df):
    """Memperoleh dataframe 1 jam berikutnya dari waktu awal s_index"
    
    Parameters:
    -----------
    s_index : Series
    df : DataFrame
        dataframe setiap shopid
    
    Returns:
    --------
    Dataframe
    """
    time_start = s_index['event_time']
    time_end = s_index['event_time'] + DateOffset(hours=1)
    mask_by_time_range = (df['event_time']>=time_start) & (df['event_time']<=time_end)
    df_by_time_range = df[mask_by_time_range]
    return df_by_time_range
    # time_start = s_index['event_time']

df_test = get_df_by_shopid(list_shopid[25], df_file)
get_df_in_a_hour_from_series(df_test.iloc[3],df_test).head()

,orderid,shopid,userid,event_time
121781,31113446916111,10311,1223238,2019-12-27 10:37:26
51555,31114127034746,10311,15372944,2019-12-27 10:48:48


In [160]:
def check_df_isSuspicious(df):
    """Memeriksa apakah dataframe rentang order 1 jam berpotensi 'order brushing'
    Nilai concentrate rate = (Banyaknya Orderan dalam 1 jam) / (Banyaknya user yang berbeda)

    Parameters:
    ----------
    df : DataFrame

    Returns:
    --------
    Boolean
        True : jika concentrate rate > =3
    """
    concentrate_rate = df.shape[0]/ (len(df['userid'].unique()))
    if concentrate_rate >= 3.0 :
        return True
    else:
        return False
        
df_test = get_df_by_shopid(list_shopid[25], df_file)
df_each_hour = get_df_in_a_hour_from_series(df_test.iloc[3],df_test)
check_df_isSuspicious(df_each_hour)

False

In [198]:
def find_suspiciousUser(df):
    """Memeriksa apakah dataframe rentang order 1 jam berpotensi 'order brushing'
    Nilai concentrate rate = (Banyaknya Orderan dalam 1 jam) / (Banyaknya user yang berbeda)

    Parameters:
    ----------
    df : DataFrame

    Returns:
    --------
    string
        "0" : Jika tidak terdapat user order brushing
        "userid" : Jika terdapat user yang melakukan order brushing
        "userid&...&useridN : Jika terdapat lebih dari satu user yang melakukan order brushing """
    
    list_user = list(df.sort_values(by=["userid"])["userid"].unique())
    list_proportion = [0 for i in range (0,len(list_user))]
    for i in range(0,len(list_user)):
        list_proportion[i] = df[df["userid"]==list_user[i]].shape[0]/df.shape[0]
    maximum_proportion = max(list_proportion)
    if(list_proportion.count(maximum_proportion)>1):
        userId_suspicious = "&".join([str(list_user[j]) for j in range(0,len(list_proportion)) if list_proportion[j]==maximum_proportion])
        # msg = "&".join([str(list_user[j]) for j in range(0,len(list_proportion) if list_proportion[j]==maximum_proportion])
        return userId_suspicious
    else:
        userId_suspicious = str(list_user[list_proportion.index(maximum_proportion)])
        return userId_suspicious


In [220]:
dict_shopId = {}
list_shopid = df_file['shopid'].unique()
for shopId in tqdm(list_shopid):
    df_shopId = get_df_by_shopid(shopId,df_file)
    df_periodSuspicious = pd.DataFrame(columns=["orderid","shopid","userid","event_time"])
    for index in range(0, df_shopId.shape[0]):
        df_each_hour = get_df_in_a_hour_from_series(df_shopId.iloc[index],df_shopId)
        if check_df_isSuspicious(df_each_hour):
            df_periodSuspicious = df_periodSuspicious.append(df_each_hour)
        else:
            pass
    if(df_periodSuspicious.shape[0]!=0):
        df_periodSuspicious = df_periodSuspicious.drop_duplicates()
        dict_shopId[str(shopId)]=find_suspiciousUser(df_periodSuspicious)
    else:
        dict_shopId[str(shopId)] = '0'

100%|██████████| 18770/18770 [03:33<00:00, 88.12it/s] 


In [221]:
with open("result.csv","w") as file:
    fieldName = ["shopid", "userid"]
    writer = csv.DictWriter(file, fieldnames= fieldName)
    writer.writeheader()
    for key,value in dict_shopId.items():
        writer.writerow({'shopid':key, 'userid':value})
